# Exploración de Datos - Versión Mejorada

Análisis conciso y accionable del dataset EXIST2025

In [ ]:
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
from pathlib import Path
import re
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from scipy.stats import chi2_contingency
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Cargar datos
with open('../lab1_materials/dataset_task1_exist2025/training.json', 'r', encoding='utf-8') as f:
    train_data = json.load(f)

df = pd.DataFrame(list(train_data.values()))
df['task1'] = df['labels_task1'].apply(lambda x: Counter(x).most_common(1)[0][0])
print(f"Muestras: {len(df)}\nClases: {df['task1'].value_counts()}")

## Análisis de Vocabulario

In [ ]:
# Análisis de vocabulario por clase
def analyze_vocabulary(df, label_col='task1'):
    results = {}
    for label in df[label_col].unique():
        texts = df[df[label_col] == label]['tweet'].tolist()
        all_words = ' '.join(texts).lower().split()
        vocab = set(all_words)
        results[label] = {
            'vocab_size': len(vocab),
            'total_words': len(all_words),
            'unique_ratio': len(vocab) / len(all_words)
        }
    return pd.DataFrame(results).T

vocab_stats = analyze_vocabulary(df)
print("Estadísticas de vocabulario por clase:")
print(vocab_stats)

## Palabras Discriminativas (Chi-square)

In [ ]:
# Palabras más discriminativas usando chi-square
vectorizer = CountVectorizer(max_features=1000, min_df=5, lowercase=True)
X = vectorizer.fit_transform(df['tweet'])
y = (df['task1'] == 'YES').astype(int)

chi2_scores = []
for i in range(X.shape[1]):
    contingency = pd.crosstab(X[:, i].toarray().ravel(), y)
    chi2, p_value, _, _ = chi2_contingency(contingency)
    chi2_scores.append((vectorizer.get_feature_names_out()[i], chi2, p_value))

chi2_df = pd.DataFrame(chi2_scores, columns=['word', 'chi2', 'p_value']).sort_values('chi2', ascending=False)
print("\nTop 20 palabras discriminativas:")
print(chi2_df.head(20).to_string(index=False))

## N-gramas Discriminativos

In [ ]:
# Análisis de bigramas por clase
def get_top_ngrams(texts, n=2, top_k=15):
    vectorizer = CountVectorizer(ngram_range=(n, n), max_features=500, min_df=3)
    X = vectorizer.fit_transform(texts)
    counts = X.sum(axis=0).A1
    ngrams = vectorizer.get_feature_names_out()
    return sorted(zip(ngrams, counts), key=lambda x: x[1], reverse=True)[:top_k]

for label in ['YES', 'NO']:
    texts = df[df['task1'] == label]['tweet'].tolist()
    bigrams = get_top_ngrams(texts, n=2, top_k=10)
    print(f"\nTop bigramas en clase {label}:")
    for ngram, count in bigrams:
        print(f"  {ngram}: {count}")

## Análisis de Patrones

In [ ]:
# Análisis de patrones textuales
patterns = {
    'urls': r'http[s]?://\S+',
    'mentions': r'@\w+',
    'hashtags': r'#\w+',
    'exclamations': r'!+',
    'questions': r'\?+',
    'caps_words': r'\b[A-Z]{2,}\b'
}

for pattern_name, pattern in patterns.items():
    df[f'has_{pattern_name}'] = df['tweet'].str.contains(pattern, regex=True)

# Correlación de patrones con etiqueta
pattern_cols = [f'has_{p}' for p in patterns.keys()]
df['label_binary'] = (df['task1'] == 'YES').astype(int)

print("\nCorrelación de patrones con sexismo:")
for col in pattern_cols:
    corr = df[[col, 'label_binary']].corr().iloc[0, 1]
    print(f"  {col}: {corr:.4f}")

## Análisis de Longitud y Complejidad

In [ ]:
# Features de complejidad
df['word_count'] = df['tweet'].str.split().str.len()
df['char_count'] = df['tweet'].str.len()
df['avg_word_len'] = df['char_count'] / df['word_count']
df['caps_ratio'] = df['tweet'].apply(lambda x: sum(1 for c in x if c.isupper()) / len(x) if len(x) > 0 else 0)

# Comparar por clase
complexity_features = ['word_count', 'char_count', 'avg_word_len', 'caps_ratio']
print("\nEstadísticas por clase:")
print(df.groupby('task1')[complexity_features].mean())

## Análisis de Agreement

In [ ]:
# Agreement vs dificultad
df['agreement'] = df['labels_task1'].apply(lambda x: Counter(x).most_common(1)[0][1] / len(x))

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Distribución de agreement
df['agreement'].hist(bins=20, ax=axes[0], edgecolor='black')
axes[0].set_title('Distribución de Agreement')
axes[0].set_xlabel('Agreement')
axes[0].set_ylabel('Frecuencia')

# Agreement por clase
df.boxplot(column='agreement', by='task1', ax=axes[1])
axes[1].set_title('Agreement por Clase')
axes[1].set_xlabel('Clase')
axes[1].set_ylabel('Agreement')

plt.tight_layout()
plt.show()

print(f"\nAgreement promedio: {df['agreement'].mean():.3f}")
print(f"Muestras con bajo agreement (<0.6): {(df['agreement'] < 0.6).sum()} ({(df['agreement'] < 0.6).mean()*100:.1f}%)")

## Resumen de Insights

In [ ]:
print("="*60)
print("INSIGHTS CLAVE")
print("="*60)
print(f"1. Desbalance de clases: {df['task1'].value_counts()['YES']/len(df)*100:.1f}% positivos")
print(f"2. Agreement promedio: {df['agreement'].mean():.2f}")
print(f"3. Vocabulario único YES/NO: {vocab_stats.loc['YES', 'vocab_size']}/{vocab_stats.loc['NO', 'vocab_size']}")
print(f"4. Palabras más discriminativas: ver chi-square")
print(f"5. Patrones importantes: URLs, mentions, exclamations")
print("="*60)